# Tutorial

## Compare Visualizer

In [ ]:
import moduletools
import numpy as np

In [ ]:
# make fake reference data
fn2 = np.arange(1*32*3*3).reshape((1, 32, 1, 3, 3)).astype(float) - 16*3*3
fn1 = fn2 + np.random.random((1, 32, 1, 3, 3)) * 0.005 - 0.0025
fn1[0][0][0] = np.nan

err_fn = "err.npz"
np.savez(err_fn, darray_actual=fn1, darray_desired=fn2)
tpu_fn = "darray_tpu_out.npz"
model_fn = "darray_model_out.npz"
np.savez(model_fn, darray=fn1)
np.savez(tpu_fn, darray=fn2)

In [ ]:
a = moduletools.NPZComparer(fn1, fn2) # npz file name, npz file, dict-like object or np.darray is OK
# a = moduletools.err_comparer(err_fn) # generate your files into one npz with ${tensor_name}_actual and ${tensor_name}_desired. Large files may be slow.
# a = moduletools.model_tpu_comparer(tpu_fn) # or
# a = moduletools.model_tpu_comparer(model_fn) # if you use tpu-mlir and want to compare model and tpu output npzs. Specify either of them.

a.info() # prints the tensor names and shape of common data
# a.ref.info() # prints the tensor names and shape of reference data
# a.target.info() # prints the tensor names and shape of target data

In [ ]:
a.compare() # do compare like npz_tool

In [ ]:
a.compare(tolerance=(0.999, 0.99)) # set tolerance to see whether all tensors have passed

In [ ]:
a.compare(tolerance=(0.9999, 0.9999), tensor='darray') # compare specific tensor(s)
# a.compare(tolerance=(0.999, 0.999), tensor=['darray'])

In [ ]:
# a.plot_target( # plot the data distribution of target data
# a.plot_ref( # plot the data distribution of reference data
# a.plot_diff( # plot the difference of target and reference
a.plot_vs( # plot the previous three things
    abs_tol=1e-8, # default 1e-8; diff within tol will be treated as no difference. 
    rel_tol=0, # default 1e-3; either tolrence pass will be treated as pass; if assigned rel_tol, values are rescaled, not original value.
    diffmin=-0.01, diffmax=0.01, # min max value of diff plot color map
    figsize=16, # the vertical size of generated figure, change it to make the figure clear. Horizontal size will be calculated automatically.
    c_columns=32, # channels on every row

#     tensor=None, # default; plot ALL the tensors. May cause OOM for large files, use with care
    tensor='darray', # tensor name, will be printed out before the figure. if np.darray, name is "darray". You can triple click the line printed out by .compare() or .info(), copy it and paste here for convinience.
#     tensor=['darray'], # plot a list of tensors

#     slices=(slice(0, 1), slice(2, 8)), # slice the ORIGINAL darray, passing slice object
#     slices=((0, 1), (2, 8)), # passing list or tuple of (end, ) or (start, end[, step])
#     slices=(0, 1), # just slice the given index, equal to slice(index, index+1)
#     slices=(0, -1), # -1 is equal to slice(None)
#     slices=(-1, None, -1, -1, -1), # use this trick to expand dim
#     slices=(-1, None, 64, slice(0, 2), (1, 2)), # you can use mixture of them

#     mix_axis=[1, 2], # for 5-or-more-dim data, merge the given dims so as to get a 4D (N, C, H, W) darray. if None, merge extra dims to C.

#     resize_hw="rectangle", # resize to nearest rectangle, no padding. If you encounter errors like "fig size too big", try this.
#     resize_hw="auto", # same to "rectangle"
#     resize_hw="square", # resize to nearest square, with padding
#     resize_hw=(-1, -1), # same to "square"
#     resize_hw=(2, 3), # resize to specific shape, with padding
#     resize_hw=(-1, 3), # resize to specific shape, self calculate h, with padding
#     resize_hw=(3, -1), # resize to specific shape, self calculate w, with padding
#     resize_hw=None, # or
#     resize_hw="none", # do not resize (default)

#     index=(2, 0), # zoom in the index in the FIGURE, idx left(right) then top(bottom)
#     transpose_hw=True # transpose the data before resize_hw. The output figure shape will also be transposed if resize_hw is none or rectangle.

#     zero_point=0., # zero point of the data color map, default None, the mean of target and ref data
#     vmin=-1, vmax=1, # min max value of data color map, default None, the min and max of target and ref data
#     dump=True, # whether do dump_vs after plot_vs (next cell)
#     verbose=True, # no use, pass to dump_vs if dump=True
)

In [ ]:
# feature: dump_vs
a.dump_vs( # same arguments as plot_vs, but print the data instead of plotting. Indexes are corresponding. Use as a reference when plot is not clear.
#     abs_tol=0.001, # default 1e-8
#     rel_tol=0.01, # default 1e-3; either tolrence pass will be treated as pass; print "!" if not pass, the number of "!" is the min of diff // tol, max to 100.
    c_columns=32, # channels on every row

#     tensor=None, # default; print ALL the tensors. May crash for large arrays, use with care
    tensor='darray', # tensor name, will be printed out before the print. if np.darray, name is "darray".
#     tensor=['darray'], # print a list of tensors

#     slices=(slice(0, 1), slice(2, 8)), # slice the ORIGINAL darray, passing slice object
#     slices=((0, 1), (2, 8)), # passing list or tuple of (end, ) or (start, end[, step])
#     slices=(0, 1), # just slice the given index, equal to slice(index, index+1)
#     slices=(0, -1), # -1 is equal to slice(None)
#     slices=(-1, None, -1, -1, -1), # use this trick to expand dim
#     slices=(-1, None, 64, slice(0, 2), (1, 2)), # you can use mixture of them

#     mix_axis=[1, 2], # for 5-or-more-d data, merge the given dims so as to get a 4D (N, C, H, W) darray. if None, merge extra dims to C.

#     resize_hw="rectangle", # resize to nearest rectangle, no padding.
#     resize_hw="auto", # same to "rectangle"
#     resize_hw="square", # resize to nearest square, with padding
#     resize_hw=(-1, -1), # same to "square"
#     resize_hw=(2, 3), # resize to specific shape, with padding
#     resize_hw=(-1, 3), # resize to specific shape, self calculate h, with padding
#     resize_hw=(3, -1), # resize to specific shape, self calculate w, with padding
#     resize_hw=None, # or
#     resize_hw="none", # do not resize (default)

    # index=(2, 0), # zoom in the index in the plot_vs figure, idx left(right) then top(bottom)
#     transpose_hw=True # transpose the data before resize_hw. The output shape will also be transposed if resize_hw is none or rectangle.

    # verbose=True # print all the indexes. If False, only print the indexes that do not pass the tolerance.
)

In [ ]:
# feature: dump_vs_plot
# when plot_vs is called, the arguments are cached and will be used by dump_vs_plot.
# you can override the arguments by passing them to dump_vs_plot.
a.dump_vs_plot()

## Float Utils

In [ ]:
from moduletools import f32, f16, bf16

In [ ]:
f32(np.pi), f16(np.pi), bf16(np.pi)

In [ ]:
f32(memory=0x40490fdb), f16(memory=0x4248), bf16(memory=0x4049)

In [ ]:
f16(10000) + f16(4)

In [ ]:
f16([0.1, 0.2, 0.3]) + f16([0.4, 0.5, 0.6])